Итак, работать будем с отзывами на кинопоиске.
Заимпортируем все необходимые библиотеки.


In [1]:
import string
import pymorphy2
import requests
import json
import re
from collections import Counter

Стоп-слова возьмем из json файла,
куда впоследствии сможем легко добавлять новые

In [2]:
with open('stopwords.json', 'r') as f:
    stopwords = json.load(f)

Отзывы будем выкачивать с помощью неофициального API
Кинопоиска. Для этого там необходимо зарегистрироваться и
добавить хэдер с ключом. Так же заранее выберем список фильмов,
у которых много отрицательных и положительных отзывов.

In [3]:
api_url = 'https://kinopoiskapiunofficial.tech/'
reviews_url = 'api/v1/reviews/'
movies = [
    '102474',
    '258687',
    '447301',
    '111543',
    '1048334',
    '692861',
    '931677',
    '9691',
    '89515',
    '263531'
]

Итак, функция, которая выкачивает рецензии на фильм.
Когда мы хотим получить слова для оценки, используем кокретный type,
а уже на рабочем анализе отзывов нам нужно отсеить только NEUTRAL отзывы

Когда мы будем качать отзывы для проверки качества нашей программы, они не должны повторяться с теми,
на которых мы учили программу, поэтому имеем параметр offset

Опыт показал, что программа показывает себя хорошо, когда учится на отзывах конкретного фильма
и делает предикшн для отзывов на него же, поэтому так и будем работать.

In [4]:
def getReviews(movie_id, amount, offset=0, type=None):
    review_list = []
    isReady = False
    params = {
        "filmId": movie_id,
    }
    headers = {
        "accept": "application/json",
        "X-API-KEY": "7c6639ff-0994-457e-9d96-81b89c205a98"
    }
    response = requests.get(api_url + reviews_url, headers=headers, params=params)
    pagesCount = response.json()['pagesCount']
    i = 1
    while (i <= pagesCount) and not isReady:
        params["page"] = i
        i += 1
        reviews = requests.get(api_url + reviews_url, headers=headers, params=params)
        reviews = reviews.json()["reviews"]
        for review in reviews:
            if len(review_list) >= (amount + offset):
                isReady = True
                break
            if type:
                if review["reviewType"] == type:
                    review_list.append(review)
            else:
                if review["reviewType"] != "NEUTRAL":
                    review_list.append(review)
    return review_list[offset:]

Теперь функция, которая обрабатывает текст отзывов и нормализует слова.
Переменная type будет нужна, когда мы захотим проверить наш предикшн.

In [5]:
def processText(reviews):
    morph = pymorphy2.MorphAnalyzer()
    normalized = []
    type = None
    for review in reviews:
        try:
            type = review["reviewType"]
            text = review["reviewDescription"].lower()
            text = text.translate(str.maketrans('', '', string.punctuation))
            text = cleantext(text)
            words = text.split(" ")
            for word in words:
                if word:
                    word = morph.parse(word)[0].normal_form
                    if not word in stopwords:
                        normalized.append(morph.parse(word)[0].normal_form)
        except:
            pass
    return normalized, type

Для чистки текста от символов, с которыми не справляется string.punctuation, используем
свою функцию

In [6]:
def cleantext(text):
    text = re.sub('[—«»…]', '', text)
    return re.sub('\r\n', ' ', text)

Функция, которая определяет, хороший отзыв или плохой, работает очень просто:
имея три массива с нормализованными словами, она просто проверяет, где больше intersection.

In [7]:
def determineReviewType(review, pos, neg):
    return "POSITIVE" if len(set(review) & set(pos)) >= len(set(review) & set(neg)) else "NEGATIVE"



Ну и наконец, собираем все вместе и проверяем на конкретном фильме.
Будем брать по 40 отзывов для обучения и проверять на 10.

In [8]:
def performCheck(movie_id):
    global totalCorrect, total
    positive_words, type = processText(getReviews(movie_id, 40, type="POSITIVE"))
    negative_words, type = processText(getReviews(movie_id, 40, type="NEGATIVE"))
    for word in positive_words:
        if word in negative_words:
            positive_words.remove(word)
            negative_words.remove(word)

    pos = [x[0] for x in Counter(positive_words).items() if x[1] >= 3]
    neg = [x[0] for x in Counter(negative_words).items() if x[1] >= 3]

    targetReviews = getReviews(movie_id, 10, offset=80)
    # print(movie_id)
    # print("----------")
    correct = 0
    for i in range(len(targetReviews)):
        reviewWords, actualType = processText([targetReviews[i]])
        probableType = determineReviewType(reviewWords, pos, neg)
        if probableType == actualType:
            # print("CORRECT", actualType)
            correct+=1
        # else:
            # print("INCORRECT", actualType, probableType)

    totalCorrect+=correct
    total+=len(targetReviews)
    # print("CORRECT -", correct, "INCORRECT -", len(targetReviews)-correct)
    # print()

Запустим цикл по всем фильмам и посмотрим, насколько правильно мы определяем, положительный отзыв или отрицательный.

In [ ]:
total = 0
totalCorrect = 0

for movie_id in movies:
    performCheck(movie_id)

print("TOTAL CORRECT -", totalCorrect, "TOTAL -", total)

Результат 74%, довольно неплохо для отзывов на Кинопоиске, которые отличаются своей
распространенностью и литературностью. Для улучшения результата можно увеличить объем обучающих выборок, отрезать более редкие слова,
обновить список стоп-слов.